In [ ]:
import ipycytoscape

In [ ]:
from pathlib import Path
from unittest.mock import patch
import sys

import wic
import wic.cli
import wic.main

testargs = ['wic', '--yaml', '']  # ignore --yaml
with patch.object(sys, 'argv', testargs):
    args = wic.cli.parser.parse_args()

tools_cwl = wic.main.get_tools_cwl(args.cwl_dirs_file)
yml_paths = wic.main.get_yml_paths(args.yml_dirs_file)

In [ ]:
import graphviz
import networkx as nx
import yaml

import wic.ast, wic.utils
from wic.schemas import wic_schema
from wic.wic_types import Yaml, YamlTree, GraphReps, NodeData, GraphData

yml_path_str = 'tutorial'
yml_namespace = 'global'
yml_path = yml_paths[yml_namespace][yml_path_str]

# Generate schemas for validation and vscode IntelliSense code completion
yaml_stems = wic.utils.flatten([list(p) for p in yml_paths.values()])
validator = wic_schema.get_validator(tools_cwl, yaml_stems, write_to_disk=True)

# Load the high-level yaml workflow file.
with open(yml_path, mode='r', encoding='utf-8') as y:
    root_yaml_tree: Yaml = yaml.safe_load(y.read())
Path('autogenerated/').mkdir(parents=True, exist_ok=True)
y_t = YamlTree(yml_path_str, root_yaml_tree)
yaml_tree_raw = wic.ast.read_ast_from_disk(y_t, yml_paths, tools_cwl, validator)
with open(f'autogenerated/{Path(yml_path).stem}_tree_raw.yml', mode='w', encoding='utf-8') as f:
    f.write(yaml.dump(yaml_tree_raw.yml))
yaml_tree = wic.ast.merge_yml_trees(yaml_tree_raw, {}, tools_cwl)
with open(f'autogenerated/{Path(yml_path).stem}_tree_merged.yml', mode='w', encoding='utf-8') as f:
    f.write(yaml.dump(yaml_tree.yml))


yaml_forest = wic.ast.tree_to_forest(yaml_tree, tools_cwl)
yaml_forest_lst =  wic.utils.flatten_forest(yaml_forest)
yaml_subtrees = dict([subforest.yaml_tree for subforest in yaml_forest_lst])
#print(yaml_subtrees.keys())


graph_gv = graphviz.Digraph(name=f'cluster_{yml_path}')
graph_gv.attr(newrank='True')
graph_nx = nx.DiGraph()
graphdata = GraphData(yml_path)
graph = GraphReps(graph_gv, graph_nx, graphdata)
compiler_info = wic.compiler.compile_workflow(yaml_tree, args, [], [graph], {}, {}, tools_cwl, True, relative_run_path=True)
rose_tree = compiler_info.rose
sub_node_data: NodeData = rose_tree.data
yaml_stem = sub_node_data.name

In [ ]:
gd = sub_node_data.graph.graphdata
gd_flat = wic.utils.flatten_graphdata(gd)
#print(len(gd_flat.nodes))
#print(len(gd_flat.edges))
json_cyto = wic.utils.graphdata_to_cytoscape(gd_flat)
#import json
#json.dumps(json_cyto)

In [ ]:
cyto = ipycytoscape.CytoscapeWidget()

cyto.graph.add_graph_from_json(json_cyto, directed=True)

import json
with open("wicStyle.json", mode='r', encoding='utf-8') as f:
    s = json.load(f)

cyto.set_style(s)
cyto.set_layout(name='cola')
cyto.box_selection_enabled = False

In [ ]:
import ipywidgets
import yaml

textarea = ipywidgets.Textarea(layout=ipywidgets.Layout(height="auto", width="auto"))
out = ipywidgets.Output()

def node_name(node):
    textarea.description = node['data'].get('label', '')
    id = node['data'].get('id', '')
    #with out:
    #    print('id', id, id in yaml_subtrees)
    #    print(yaml_subtrees.get(id))
    textarea.value = yaml.dump(yaml_subtrees.get(id, ''))

# TODO: Round tripping: After user edits AST in textarea, re-compile.

cyto.on('node', 'click', node_name)

In [ ]:
from IPython.display import display
display(cyto, textarea)

In [ ]:
display(out)